## Predicting Spectrum Traits

In [1]:
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torch
import os
import HyperSpectrumDataSet as hsd
import torch.optim as optim
import HyperSpectrumModel as model
import torch.nn as nn
import torch.nn.functional as F
from tensorboardX import SummaryWriter
import sys
import datetime
import copy
from sklearn.preprocessing import scale 

In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print("... runing on .. ", device)

... runing on ..  cuda


In [3]:
best_paras = {}

best_paras["LMA_O"] = [0.0005, 0.5, 0.0003, 0.0] 
best_paras["Narea_O"] = [0.0005, 0.0, 0.0001, 0.0] 
best_paras["SPAD_O"] = [0.0005, 0.3, 0.0, 0.05] 
best_paras["Nmass_O"] = [0.0005, 0.5, 0.0003, 0.0]
best_paras["Pmass_O"] = [0.0005, 0.7, 0.0005, 0.0] 
best_paras["Vcmax"] = [0.0001, 0.3, 0.0005, 0.0] 
best_paras["Vcmax25"] = [0.001, 0.3, 0.0003, 0.0] 
best_paras["J"] = [0.0001, 0.7, 0.0, 0.0] 
best_paras["Photo_O"] = [0.0005, 0.5, 0.0003, 0.01] 
best_paras["Cond_O"] = [0.0005, 0.7, 0.0003, 0.0]


In [4]:
use_best_params = True

drop_percent = 0
exp_desc = "HyperSpectrumModel4Layers_optimized_"+str(drop_percent)

now = datetime.datetime.now()

script_dir = os.path.dirname("__file__")

result_file_name = os.path.join(script_dir, "results")
result_file_name = os.path.join(result_file_name, exp_desc + "_" + now.strftime("%Y-%m-%d-%H-%M")+"_results.txt")

log_dir = os.path.join(script_dir, "runs")
log_dir = os.path.join(log_dir, exp_desc + "_" + now.strftime("%Y-%m-%d-%H-%M"))

writer = SummaryWriter(log_dir=log_dir)

all_traits = ["LMA_O", "Narea_O", "SPAD_O", "Nmass_O", "Pmass_O", "Vcmax", "Vcmax25", "J", "Photo_O", "Cond_O"]


run_traits = all_traits

early_stoppings = 200

num_epochs = 1000
batch_size = 64
test_every = 10
lr = 0.01
dropout_percent = 0
weight_decay = 0
stddev = 0

test_results = {}
train_results = {}
val_results = {}
for target_trait in run_traits:

    train_ds = hsd.HyperSpectrumDataSet(target_trait, "train", drop_percent=drop_percent)
        
    trainloader = torch.utils.data.DataLoader(train_ds, batch_size=batch_size,
                                              shuffle=True)

    test_ds = hsd.HyperSpectrumDataSet(target_trait, "test")
    testloader = torch.utils.data.DataLoader(test_ds, batch_size=batch_size,
                                              shuffle=False)
    val_ds = hsd.HyperSpectrumDataSet(target_trait, "val")
    validationloader = torch.utils.data.DataLoader(val_ds, batch_size=batch_size,
                                              shuffle=False)
    min_y = min(train_ds.data_labels[target_trait])
    max_y = max(train_ds.data_labels[target_trait])
    
    if use_best_params:
        lr = best_paras[target_trait][0]
        dropout_percent = best_paras[target_trait][1]
        weight_decay = best_paras[target_trait][2]
        stddev = best_paras[target_trait][3]
        print(best_paras[target_trait])
    else:
        lr = 0.01
        dropout_percent = 0
        weight_decay = 0
        stddev = 0
        
    hyperSpectrumModel = model.HyperSpectrumModel4Layers(dropout_percent=dropout_percent,stddev=stddev)

    
    criterion = nn.MSELoss()
    optimizer = optim.Adam(hyperSpectrumModel.parameters(), lr=lr, weight_decay=weight_decay)
    
    hyperSpectrumModel.to(device)
    print(hyperSpectrumModel)
    
    sum = 0
    for i in range(len(train_ds)):
        values, labels = train_ds[i]
        sum += labels

    train_label_mean = sum / len(train_ds)
    print("train label mean: " + str(train_label_mean))

    train_label_sq_diff = 0
    for i in range(len(train_ds)):
        values, labels = train_ds[i]
        train_label_sq_diff += (labels -  train_label_mean) **2

    sum = 0
    for i in range(len(val_ds)):
        values, labels = val_ds[i]
        sum += labels

    val_label_mean = sum / len(val_ds)
    print("validation label mean: " + str(val_label_mean))

    val_label_sq_diff = 0
    for i in range(len(val_ds)):
        values, labels = val_ds[i]
        val_label_sq_diff += (labels -  val_label_mean) **2


    sum = 0
    for i in range(len(test_ds)):
        values, labels = test_ds[i]
        sum += labels

    test_label_mean = sum / len(test_ds)
    print("test label mean: " + str(test_label_mean))

    test_label_sq_diff = 0
    for i in range(len(test_ds)):
        values, labels = test_ds[i]
        test_label_sq_diff += (labels -  test_label_mean) **2

    bestModel = copy.deepcopy(hyperSpectrumModel)
    best_validation_rsquare = -99999.99
    
    Train_R_square_value = 0
    not_improved_count = 0

    for epoch in range(num_epochs):
        running_loss = 0.0
        pred_sq_diff = 0.0
        hyperSpectrumModel.train()
        for i, data in enumerate(trainloader, 0):
            values, labels = data            
            labels = labels.unsqueeze(1)
            
            # run on gpu if available
            values = values.to(device)
            labels = labels.to(device)
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = hyperSpectrumModel(values)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            # print statistics
            running_loss += loss.item()
            diff = outputs - labels
            pred_sq_diff += torch.sum(torch.mul(diff, diff),0).detach().cpu().numpy()

        Train_R_square_value = 1 - pred_sq_diff / train_label_sq_diff
        
        if ((epoch % test_every) == 0):
            print("Epoch: ", epoch, " Trait: ", target_trait, ", Train_R_square_value: ", np.asscalar(Train_R_square_value))
            running_loss = 0.0
            pred_sq_diff = 0.0
            hyperSpectrumModel.eval()
            for i, data in enumerate(validationloader, 0):
                values, labels = data
                labels = labels.unsqueeze(1)
                
                # run on gpu if available
                values = values.to(device)
                labels = labels.to(device)
            
                outputs = hyperSpectrumModel(values)
                outputs[outputs < min_y] = min_y
                outputs[outputs > max_y] = max_y
                    
                loss = criterion(outputs, labels)
                running_loss += loss.item()
                diff = outputs - labels
                pred_sq_diff += torch.sum(torch.mul(diff, diff), 0).detach().cpu().numpy()

            writer.add_scalar(target_trait + '/training_Rsquare', np.asscalar(Train_R_square_value), epoch)
            Validation_R_square_value = 1 - pred_sq_diff / val_label_sq_diff
            writer.add_scalar(target_trait + '/validation_Rsquare', np.asscalar(Validation_R_square_value), epoch)
            print("validation rsquare: " + str(np.asscalar(Validation_R_square_value)))
            # copy the best model so far
            if (best_validation_rsquare < Validation_R_square_value):
                print("copying best model")
                bestModel = copy.deepcopy(hyperSpectrumModel)
                best_validation_rsquare = Validation_R_square_value
                not_improved_count = 0 # restart
            else:
                not_improved_count += 1
                if not_improved_count == early_stoppings:
                    print ("the model is not improved, stop!")
                    break

    train_results[target_trait] = Train_R_square_value
    val_results[target_trait] = best_validation_rsquare
    print("Trait: ", target_trait, ", Best Val_R_square_value: ", np.asscalar(best_validation_rsquare))
    num_iters = 0
    running_loss = 0.0
    pred_sq_diff = 0.0

    bestModel.eval()
    for i, data in enumerate(testloader, 0):
        values, labels = data        
        labels = labels.unsqueeze(1)
        # run on gpu if available
        values = values.to(device)
        labels = labels.to(device)
        outputs = bestModel(values)
        
        loss = criterion(outputs, labels)
        running_loss += loss.item()
        num_iters += 1
        diff = outputs - labels
        pred_sq_diff += torch.sum(torch.mul(diff, diff), 0).detach().cpu().numpy()

    
    
    print("Trait: ", target_trait, ", test loss: ", running_loss / num_iters)
    Test_R_square_value = 1 - pred_sq_diff / test_label_sq_diff
    print("Trait: ", target_trait, ", Test_R_square_value: ", np.asscalar(Test_R_square_value))
    test_results[target_trait] = Test_R_square_value

writer.close()

result_file = open(result_file_name,"w")
result_file.write(str(bestModel)+"\n")
result_file.write("lr: "+ str(lr) + ", dropout: " + str(dropout_percent) + ", weight_decay:" + str(weight_decay)+"\n")
result_file.write("-------------------------------------------------\n")
result_file.write("Here are the training data set results:\n")
total = 0.0
for key, value in train_results.items():
    result_file.write("trait: " + str(key) +", R square: " + str(np.asscalar(value))+"\n")
    total += np.asscalar(value)
result_file.write("Average R square: " + str(total / len(train_results))+"\n")
result_file.write("-------------------------------------------------\n")
result_file.write("Here are the validation data set results:\n")
total = 0.0
for key, value in val_results.items():
    result_file.write("trait: " + str(key) +", R square: " + str(np.asscalar(value))+"\n")
    total += np.asscalar(value)
result_file.write("Average R square: " + str(total / len(train_results))+"\n")
result_file.write("-------------------------------------------------\n")
result_file.write("Here are the testing data set results: \n")
total = 0.0
for key, value in test_results.items():
    result_file.write("trait: " + str(key) + ", R square: " + str(np.asscalar(value))+"\n")
    total += np.asscalar(value)
result_file.write("Average R square: " + str(total / len(test_results)) + "\n")
result_file.write("-------------------------------------------------\n")


result_file = open(result_file_name+".csv","w")
result_file.write("Model, Dataset, LMA_O, Narea_O, SPAD_O, Nmass_O, Parea_O, Pmass_O, Vcmax, Vcmax25, J, Photo_O, Cond_O, Average\n")

result_file.write(exp_desc +", train, ") 
total = 0.0

for trait in run_traits:
    if trait in train_results:
        result_file.write(str(np.asscalar(value))+", ")
        total += np.asscalar(value)
    else:
        result_file.write(", ")

result_file.write(str(total)+"\n")

result_file.write(exp_desc +", val, ") 
total = 0.0

for trait in run_traits:
    if trait in val_results:
        result_file.write(str(np.asscalar(value))+", ")
        total += np.asscalar(value)
    else:
        result_file.write(", ")

result_file.write(str(total)+"\n")

result_file.write(exp_desc +", test, ") 
total = 0.0

for trait in run_traits:
    if trait in test_results:
        result_file.write(str(np.asscalar(value))+", ")
        total += np.asscalar(value)
    else:
        result_file.write(", ")

result_file.write(str(total)+"\n")


result_file.close()


[0.0005, 0.5, 0.0003, 0.0]
HyperSpectrumModel4Layers(
  (noise): GaussianNoise()
  (avgPool1): AvgPool1d(kernel_size=(5,), stride=(5,), padding=(0,))
  (block1): DeepBasicBlock(
    (Conv1D): Conv1d(1, 50, kernel_size=(5,), stride=(5,), padding=(1,))
    (bn): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
  )
  (block2_1): DeepBasicBlock(
    (Conv1D): Conv1d(50, 100, kernel_size=(3,), stride=(3,))
    (bn): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
  )
  (block2_2): DeepBasicBlock(
    (Conv1D): Conv1d(100, 200, kernel_size=(3,), stride=(3,))
    (bn): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
  )
  (block3_1): DeepBasicBlock(
    (Conv1D): Conv1d(200, 400, kernel_size=(3,), stride=(3,))
    (bn): BatchNorm1d(400, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inpl

Epoch:  600  Trait:  LMA_O , Train_R_square_value:  0.953213632106781
validation rsquare: 0.8569518327713013
Epoch:  610  Trait:  LMA_O , Train_R_square_value:  0.9516757726669312
validation rsquare: 0.8785315752029419
Epoch:  620  Trait:  LMA_O , Train_R_square_value:  0.9490011930465698
validation rsquare: 0.8497793674468994
Epoch:  630  Trait:  LMA_O , Train_R_square_value:  0.963263750076294
validation rsquare: 0.8437638282775879
Epoch:  640  Trait:  LMA_O , Train_R_square_value:  0.9595347046852112
validation rsquare: 0.8208311796188354
Epoch:  650  Trait:  LMA_O , Train_R_square_value:  0.9530799388885498
validation rsquare: 0.7612134218215942
Epoch:  660  Trait:  LMA_O , Train_R_square_value:  0.9518113136291504
validation rsquare: 0.8335537910461426
Epoch:  670  Trait:  LMA_O , Train_R_square_value:  0.9575941562652588
validation rsquare: 0.7874505519866943
Epoch:  680  Trait:  LMA_O , Train_R_square_value:  0.9603635668754578
validation rsquare: 0.8112204074859619
Epoch:  690 

Epoch:  190  Trait:  Narea_O , Train_R_square_value:  0.9678475856781006
validation rsquare: 0.732967734336853
Epoch:  200  Trait:  Narea_O , Train_R_square_value:  0.9646916389465332
validation rsquare: 0.9261229634284973
copying best model
Epoch:  210  Trait:  Narea_O , Train_R_square_value:  0.9687914252281189
validation rsquare: 0.785100519657135
Epoch:  220  Trait:  Narea_O , Train_R_square_value:  0.9696774482727051
validation rsquare: 0.4876883029937744
Epoch:  230  Trait:  Narea_O , Train_R_square_value:  0.9558001160621643
validation rsquare: 0.8924469947814941
Epoch:  240  Trait:  Narea_O , Train_R_square_value:  0.9594693779945374
validation rsquare: 0.8257503509521484
Epoch:  250  Trait:  Narea_O , Train_R_square_value:  0.9698830842971802
validation rsquare: 0.8774347305297852
Epoch:  260  Trait:  Narea_O , Train_R_square_value:  0.9769397377967834
validation rsquare: 0.9013047814369202
Epoch:  270  Trait:  Narea_O , Train_R_square_value:  0.9735451936721802
validation rsq

Epoch:  920  Trait:  Narea_O , Train_R_square_value:  0.9935217499732971
validation rsquare: 0.8880642652511597
Epoch:  930  Trait:  Narea_O , Train_R_square_value:  0.9936693906784058
validation rsquare: 0.8876345157623291
Epoch:  940  Trait:  Narea_O , Train_R_square_value:  0.9926174283027649
validation rsquare: 0.8953854441642761
Epoch:  950  Trait:  Narea_O , Train_R_square_value:  0.9936415553092957
validation rsquare: 0.8678591847419739
Epoch:  960  Trait:  Narea_O , Train_R_square_value:  0.9933720231056213
validation rsquare: 0.8999924063682556
Epoch:  970  Trait:  Narea_O , Train_R_square_value:  0.9959707856178284
validation rsquare: 0.8214693069458008
Epoch:  980  Trait:  Narea_O , Train_R_square_value:  0.994796633720398
validation rsquare: 0.9222054481506348
Epoch:  990  Trait:  Narea_O , Train_R_square_value:  0.9960487484931946
validation rsquare: 0.9079066514968872
Trait:  Narea_O , Best Val_R_square_value:  0.9312006235122681
Trait:  Narea_O , test loss:  0.1207231655

Epoch:  510  Trait:  SPAD_O , Train_R_square_value:  0.8642450571060181
validation rsquare: 0.876285195350647
Epoch:  520  Trait:  SPAD_O , Train_R_square_value:  0.8677736520767212
validation rsquare: 0.8928249478340149
copying best model
Epoch:  530  Trait:  SPAD_O , Train_R_square_value:  0.8741870522499084
validation rsquare: 0.8833332061767578
Epoch:  540  Trait:  SPAD_O , Train_R_square_value:  0.8675929307937622
validation rsquare: 0.8770412802696228
Epoch:  550  Trait:  SPAD_O , Train_R_square_value:  0.8713947534561157
validation rsquare: 0.8552187085151672
Epoch:  560  Trait:  SPAD_O , Train_R_square_value:  0.8529588580131531
validation rsquare: 0.8660843372344971
Epoch:  570  Trait:  SPAD_O , Train_R_square_value:  0.8685631155967712
validation rsquare: 0.8652321100234985
Epoch:  580  Trait:  SPAD_O , Train_R_square_value:  0.8605822920799255
validation rsquare: 0.875845193862915
Epoch:  590  Trait:  SPAD_O , Train_R_square_value:  0.8695906400680542
validation rsquare: 0.8

...

validation label mean: -0.10102464099015508
test label mean: -0.11235753853910978
Epoch:  0  Trait:  J , Train_R_square_value:  0.007365107536315918
validation rsquare: -0.02286672592163086
copying best model
Epoch:  10  Trait:  J , Train_R_square_value:  0.487731397151947
validation rsquare: 0.49926185607910156
copying best model
Epoch:  20  Trait:  J , Train_R_square_value:  0.5771255493164062
validation rsquare: 0.7949803471565247
copying best model
Epoch:  30  Trait:  J , Train_R_square_value:  0.6620374917984009
validation rsquare: 0.8204519152641296
copying best model
Epoch:  40  Trait:  J , Train_R_square_value:  0.6971791982650757
validation rsquare: 0.8068345785140991
Epoch:  50  Trait:  J , Train_R_square_value:  0.7167544364929199
validation rsquare: 0.8327758312225342
copying best model
Epoch:  60  Trait:  J , Train_R_square_value:  0.729102373123169
validation rsquare: 0.8136095404624939
Epoch:  70  Trait:  J , Train_R_square_value:  0.73616623878479
validation rsquare: 0.

Epoch:  750  Trait:  J , Train_R_square_value:  0.9131112098693848
validation rsquare: 0.8559892177581787
Epoch:  760  Trait:  J , Train_R_square_value:  0.9277288317680359
validation rsquare: 0.8653271198272705
Epoch:  770  Trait:  J , Train_R_square_value:  0.9242726564407349
validation rsquare: 0.853854775428772
Epoch:  780  Trait:  J , Train_R_square_value:  0.9147903919219971
validation rsquare: 0.8735157251358032
Epoch:  790  Trait:  J , Train_R_square_value:  0.9196081161499023
validation rsquare: 0.8767095804214478
Epoch:  800  Trait:  J , Train_R_square_value:  0.94603431224823
validation rsquare: 0.8663696050643921
Epoch:  810  Trait:  J , Train_R_square_value:  0.9291765093803406
validation rsquare: 0.8756120800971985
Epoch:  820  Trait:  J , Train_R_square_value:  0.9126411080360413
validation rsquare: 0.8390333652496338
Epoch:  830  Trait:  J , Train_R_square_value:  0.9276567697525024
validation rsquare: 0.8545753955841064
Epoch:  840  Trait:  J , Train_R_square_value:  0

Epoch:  340  Trait:  Photo_O , Train_R_square_value:  0.69438636302948
validation rsquare: 0.6307681202888489
Epoch:  350  Trait:  Photo_O , Train_R_square_value:  0.7014564871788025
validation rsquare: 0.6890890598297119
Epoch:  360  Trait:  Photo_O , Train_R_square_value:  0.7169811725616455
validation rsquare: 0.6308778524398804
Epoch:  370  Trait:  Photo_O , Train_R_square_value:  0.7485306262969971
validation rsquare: 0.6885813474655151
Epoch:  380  Trait:  Photo_O , Train_R_square_value:  0.722369909286499
validation rsquare: 0.6741461753845215
Epoch:  390  Trait:  Photo_O , Train_R_square_value:  0.6949039697647095
validation rsquare: 0.7032917737960815
copying best model
Epoch:  400  Trait:  Photo_O , Train_R_square_value:  0.7103382349014282
validation rsquare: 0.673046350479126
Epoch:  410  Trait:  Photo_O , Train_R_square_value:  0.7453900575637817
validation rsquare: 0.6463534832000732
Epoch:  420  Trait:  Photo_O , Train_R_square_value:  0.7669442296028137
validation rsqua

validation label mean: 0.06509911896166753
test label mean: -0.11372655154129048
Epoch:  0  Trait:  Cond_O , Train_R_square_value:  0.009232699871063232
validation rsquare: -0.016027092933654785
copying best model
Epoch:  10  Trait:  Cond_O , Train_R_square_value:  0.20185178518295288
validation rsquare: -0.008385181427001953
copying best model
Epoch:  20  Trait:  Cond_O , Train_R_square_value:  0.25909602642059326
validation rsquare: 0.12933707237243652
copying best model
Epoch:  30  Trait:  Cond_O , Train_R_square_value:  0.37091922760009766
validation rsquare: -0.025893688201904297
Epoch:  40  Trait:  Cond_O , Train_R_square_value:  0.4127245545387268
validation rsquare: 0.1976218819618225
copying best model
Epoch:  50  Trait:  Cond_O , Train_R_square_value:  0.4418039917945862
validation rsquare: 0.1363515853881836
Epoch:  60  Trait:  Cond_O , Train_R_square_value:  0.3639281392097473
validation rsquare: 0.15396881103515625
Epoch:  70  Trait:  Cond_O , Train_R_square_value:  0.5008

Epoch:  710  Trait:  Cond_O , Train_R_square_value:  0.876451849937439
validation rsquare: -1.6896090507507324
Epoch:  720  Trait:  Cond_O , Train_R_square_value:  0.8351085186004639
validation rsquare: -0.3244084119796753
Epoch:  730  Trait:  Cond_O , Train_R_square_value:  0.8650667071342468
validation rsquare: -0.3227701187133789
Epoch:  740  Trait:  Cond_O , Train_R_square_value:  0.8415851593017578
validation rsquare: 0.12847411632537842
Epoch:  750  Trait:  Cond_O , Train_R_square_value:  0.8676701784133911
validation rsquare: 0.12459415197372437
Epoch:  760  Trait:  Cond_O , Train_R_square_value:  0.8815208077430725
validation rsquare: -0.326543927192688
Epoch:  770  Trait:  Cond_O , Train_R_square_value:  0.8881191611289978
validation rsquare: 0.1711239218711853
Epoch:  780  Trait:  Cond_O , Train_R_square_value:  0.8734372854232788
validation rsquare: -0.7351276874542236
Epoch:  790  Trait:  Cond_O , Train_R_square_value:  0.8699410557746887
validation rsquare: 0.3050284385681

best parameters
all_traits = ["LMA_O", "Narea_O", "SPAD_O", "Nmass_O", "Pmass_O", "Vcmax", "Vcmax25", "J", "Photo_O", "Cond_O"]
